In [10]:
import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam


In [11]:
!pip install datasets

In [12]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


In [13]:
from datasets import Dataset, DatasetDict
import pandas as pd

# 📌 CSV dosyalarını yükle
train_df = pd.read_csv("dataset/translated_train.csv")  # Eğitim veri seti
test_df = pd.read_csv("dataset/translated_test.csv")    # Test veri seti

# Pandas DataFrame'leri Hugging Face Dataset formatına çevirme
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 📌 Eğitim veri setinin tamamını kullanmak istersen:
# combined_datasets = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })

# Hugging Face DatasetDict oluşturma
combined_datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Sonuçları kontrol et
print(combined_datasets)

README.md:   0%|          | 0.00/614 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/686M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def tokenize_sample_data(data):

    input_feature = tokenizer(data['article'], truncation=True, max_length=1024)
    label = tokenizer(data['highlights'], truncation=True, max_length=128)
    return {
        "input_ids" : input_feature['input_ids'],
        "attention_mask" : input_feature['attention_mask'],
        "labels" : label['input_ids'],
    }

tokenized_ds = combined_datasets.map(
    tokenize_sample_data,
    batched=True,
    batch_size= 56
)
tokenized_ds

Map:   0%|          | 0/92524 [00:00<?, ? examples/s]

Map:   0%|          | 0/14610 [00:00<?, ? examples/s]

Map:   0%|          | 0/15379 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 92524
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14610
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15379
    })
})

In [ ]:
'''

from transformers import MT5Tokenizer, MT5ForConditionalGeneration

# 1. Tokenizer ve modelin yüklenmesi
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 2. Özetlenmek istenen Türkçe metin
turkish_text = """
İklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.
Özellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.
Bu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.
"""

# 3. Özetleme için giriş formatı: 'summarize:' ön ekini ekliyoruz
input_text = f"summarize: {turkish_text}"

# 4. Tokenizer ile metni encode etme
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Modeli kullanarak özetleme yapma
summary_ids = model.generate(
    input_ids,
    max_length=150,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# 6. Özetlenen metni decode etme
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 7. Sonucu yazdırma
print("Özetlenen Metin:")
print(summary)
'''

'\n\nfrom transformers import MT5Tokenizer, MT5ForConditionalGeneration\n\n# 1. Tokenizer ve modelin yüklenmesi\nmodel_name = "google/mt5-small"\ntokenizer = MT5Tokenizer.from_pretrained(model_name)\nmodel = MT5ForConditionalGeneration.from_pretrained(model_name)\n\n# 2. Özetlenmek istenen Türkçe metin\nturkish_text = """\nİklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.\nÖzellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.\nBu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.\n"""\n\n# 3. Özetleme için giriş formatı: \'summarize:\' ön ekini ekliyoruz\ninput_text = f"summarize: {turkish_text}"\n\n# 4. Tokenizer ile metni encode etme\ninput_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n\n# 5. Modeli kullanarak özetleme yapma\nsummary_ids = model.genera

In [ ]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model konfigürasyonu
mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=2056,
    length_penalty=1.0,
    no_repeat_ngram_size=3,
    num_beams=5,
)

# PyTorch tabanlı modeli yükleme ve GPU/CPU'ya taşıma
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small", config=mt5_config).to(device)


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  return_tensors="pt")

In [ ]:
!pip install evaluate
!pip install rouge_score

In [ ]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    return arg  # Bu fonksiyon kaldırılabilir veya değiştirilmeden kullanılabilir

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith((".", "!", "?")) else p + ".") for p in text_preds]
  text_labels = [(l if l.endswith((".", "!", "?")) else l + ".") for l in text_labels]

  # Türkçe cümle tokenizasyonu
  sent_tokenizer_tr = RegexpTokenizer(r'[^.!?]*[.!?]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(l))) for l in text_labels]

  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-summarize-tr",
    log_level="error",
    num_train_epochs=5,  # Küçük veri seti için 5 epoch yeterli
    learning_rate=2e-5,  # 3e-5 fazla olabilir, 2e-5 daha stabil
    lr_scheduler_type="linear",
    warmup_steps=1000,
    optim="adafactor",
    weight_decay=0.01,
    per_device_train_batch_size=4,  # 16 yerine 8 (bellek tasarrufu)
    per_device_eval_batch_size=4,   # 8 yerine 4 (bellek tasarrufu)
    gradient_accumulation_steps=8,  # 4 yerine 8 (küçük batch size için telafi)
    evaluation_strategy="steps",
    eval_steps=250,
    predict_with_generate=True,
    generation_max_length=256,
    save_steps=2000,  # 1000 yerine 2000 (Bellek optimizasyonu)
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    #fp16=True  # 🔥 Mixed Precision Training (Bellek kullanımı azalır)
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

# EarlyStoppingCallback ile erken durdurma tanımlıyoruz
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=4,  # Art arda kaç kez validation loss'un düşmemesine izin verilecek
    early_stopping_threshold=0.0  # Minimum değişiklik miktarı (opsiyonel)
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    callbacks=[early_stopping]  # Erken durdurma callback'i buraya ekleniyor
)

trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 372.88 MiB is free. Process 31822 has 21.80 GiB memory in use. Of the allocated memory 21.51 GiB is allocated by PyTorch, and 67.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Eğitilen modeli kaydet
output_dir = "./trained_model"  # Kaydedilecek dizin
trainer.save_model(output_dir)  # Modeli ve tokenizer'ı kaydeder

# Tokenizer'ı ayrıca kaydetmek isterseniz
tokenizer.save_pretrained(output_dir)

print(f"Model ve tokenizer {output_dir} dizinine kaydedildi.")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Eğitilen modeli ve tokenizer'ı yükle
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")


In [ ]:
test_text = "Dünyamızda 1900’lü yıllardan günümüze kadar dil öğretiminde çeşitli metinler kullanılmıştır. Bunlar “ edebi metinler, üretilmiş metinler, özgün ve özel metinler ” başlıkları altında toplanmıştır. Metinlerin seçimi dil öğretim yaklaşım ve yöntemlerine göre değişmektedir. Her yaklaşım kendine özgü metin kullanmıştır. Geleneksel yaklaşımda dil bilgisi kuralları, atasözleri, edebiyat, genel kültür gibi konuların öğretimine ağırlık verildiğinden edebi metinler kullanılmıştır. Davranışçı yaklaşımda dil davranış olarak ele alınmış, tekrar, taklit ve ezberleme yoluyla öğretilmiştir. Bu yaklaşımda edebi metinler yerine üretilmiş metinler kullanılmıştır. Bilişsel yaklaşımda “dil iletişim aracıdır” görüşü yayılmış ve özgün metinler kullanılmaya başlanmıştır. "


In [ ]:
'''inputs = tokenizer(
    test_text,
    return_tensors="pt",  # PyTorch tensörleri olarak döner
    max_length=1024,      # Modelin giriş sınırı
    truncation=True       # Çok uzun metinleri keser
)
'''

In [ ]:
# Özetleme
output_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,  # Özet uzunluğu sınırı
    num_beams=4,     # Beam search kullanımı
    no_repeat_ngram_size=2,  # Tekrar eden n-gramları engeller
    length_penalty=0.6       # Kısa özetlere öncelik
)

# Özet çözümleme
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Özet:", summary)


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''